### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5233,2023-10-23,China Cba,08:35,Beijing Royal Fighters,Qingdao,1.69,2.15,191.5,1.87,1.87,-1.5,1.80,1.95
5234,2023-10-23,China Cba,08:35,Guangzhou,Shandong,1.61,2.35,189.5,1.87,1.87,-2.5,1.83,1.91
5235,2023-10-23,China Cba,08:35,Jiangsu Dragons,Xinjiang,4.60,1.20,195.5,1.87,1.87,7.5,1.95,1.80
5236,2023-10-23,Coreia Do Sul Kbl,07:00,Mobis Phoebus,LG Sakers,1.48,2.75,159.5,1.91,1.91,-6.5,1.95,1.87
5237,2023-10-23,Grécia Liga De Basquete,11:15,Promitheas,Marousi,1.17,5.50,155.5,1.91,1.91,-10.5,1.91,1.91
5238,2023-10-23,Grécia Liga De Basquete,14:15,Panathinaikos,Lavrio,1.04,15.00,156.5,1.87,1.95,-17.5,1.95,1.87


## Apostas do dia

### Over v1 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,07:00,Coreia Do Sul Kbl,Mobis Phoebus,LG Sakers,159.5,1.91,0.7390,Over
4,11:15,Grécia Liga De Basquete,Promitheas,Marousi,155.5,1.91,0.8439,Over
5,14:15,Grécia Liga De Basquete,Panathinaikos,Lavrio,156.5,1.87,0.7662,Over


### Over v1 (LDA)

In [9]:
# Duplicando o dataset
df_over_v1 = jogos.copy()

df_over_v1.reset_index(drop=True, inplace=True)
df_over_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v1.empty:
    mdl_over_v1 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v1, df_over_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V1 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V1 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,11:15,Grécia Liga De Basquete,Promitheas,Marousi,155.5,1.91,0.8439,Over
5,14:15,Grécia Liga De Basquete,Panathinaikos,Lavrio,156.5,1.87,0.7662,Over


### Over v2 (LDA)

In [10]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['Over_Line'] < 225))]
    prev = prev[((prev['P_Diff'] < 0.043) | (prev['P_Diff'] > 0.045))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,11:15,Grécia Liga De Basquete,Promitheas,Marousi,155.5,1.91,0.6341,Over


### Over v3 (Blend)

In [11]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds_A'] < 1.75) | (prev['Odds_A'] > 2.8))]
    prev = prev[((prev['HA_Line'] < -8.5) | (prev['HA_Line'] > -5.5))]
    prev = prev[((prev['CV_ML'] >= 0) & (prev['CV_ML'] < 0.65)) | ((prev['CV_ML'] >= 0.77) & (prev['CV_ML'] < 0.87)) | ((prev['CV_ML'] >= 0.95) & (prev['CV_ML'] < 1.5))]


    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (BLEND) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (BLEND) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,Bet
5,14:15,Grécia Liga De Basquete,Panathinaikos,Lavrio,156.5,1.87,Over
